In [1]:
# Chapter 6

In [5]:
# Classification with KNN

In [1]:
from sklearn.datasets import load_iris
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors

In [2]:
iris = load_iris()

In [3]:
X = iris.data
Y = iris.target

In [4]:
XTrain, XTest, YTrain, YTest =\
ms.train_test_split(X,Y, test_size = 0.3, random_state = 7)

In [5]:
# With KNN we need to find the appropriate value of k. and we can do this 
# with the help of GridSearchCV

In [19]:
k_neighbours = list(range(1,21,2)) # list of even numbers as candidates
n_grid = [{'n_neighbors':k_neighbours}]

In [20]:
model = neighbors.KNeighborsClassifier()

In [21]:
# We are performing cross-validation with 10 folds in our training data

In [25]:
cv_knn = GridSearchCV(estimator = model, param_grid =  n_grid,\
                     cv = ms.KFold(n_splits=10))

In [26]:
cv_knn.fit(XTrain, YTrain)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid=[{'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [27]:
best_k = cv_knn.best_params_['n_neighbors']

In [28]:
print('The best parameter is k = {0}'.format(best_k))

The best parameter is k = 11


In [31]:
# For visualisation purposes we will concentrate on only two features:
# sepal width and petal lenght

knnclf = neighbors.KNeighborsClassifier(n_neighbors=best_k)

knnclf.fit(XTrain[:, 2:4], YTrain) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=11, p=2,
           weights='uniform')

In [32]:
y_pred = knnclf.predict(XTest[:, 2:4])

In [33]:
from sklearn.metrics import confusion_matrix

In [34]:
confusion_matrix(YTest, y_pred)

array([[12,  0,  0],
       [ 0, 14,  2],
       [ 0,  2, 15]])

In [35]:
# 4 instances have been missclassified. 

In [36]:
from sklearn.metrics import classification_report

In [37]:
print(classification_report(YTest, y_pred))

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        12
          1       0.88      0.88      0.88        16
          2       0.88      0.88      0.88        17

avg / total       0.91      0.91      0.91        45

